<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Visualização Interativa de Dados
Caderno de **Aula**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Google Data Studio;</li>
  <li>Dados;</li>
  <li>Relatório.</li>
</ol>

---

# **Aulas**

## 1\. Google Data Studio

### **1.1. Inteligência de negócio**

> Inteligência de negócio, do inglês *Business Intelligenc*e (BI), é uma especialidade da análise de dados que busca facilitar o acesso a informação para tomada de decisão através de ferramentas de visualização interativa de dados.

Um dos objetivos da área é a definição (e exposição) de **indicadores chaves de desempenho** (*key performance indicato*r ou KPI) e facilitação do uso da **análise exploratória de dados** (*exploratory data analysis* ou EDA) de forma interativa para usuário de negócio (geralmente não técnicos) através do uso de ferramentas de BI.

### **1.2. Ferramentas de BI**

As ferramentas líderes de mercado são o [Tableau](https://www.tableau.com) e o [Microsoft Power BI](https://powerbi.microsoft.com/), ambas pagas com uma limitada versão gratuita. Neste curso utilizaremos o [Google Data Studio](https://datastudio.google.com/) que apresenta uma versão completa e gratuita.

### **1.3. Termos técnicos**

Diferentes ferramentas de visualização interativa de dados utilizam um conjunto de termos técnicos comun para construção de painéis de dados:

 - **Relatório** (*report*): o painel de dados (*dashboard*);
 - **Fonte de dados** (*data source*): conjunto de dados usado nas visualizações de dados do painel;
 - **Dimensões** (*dimensions*): coluna do conjunto de dados;
 - **Métricas** (*metrics*): operação de agregação aplicada a uma ou mais dimensões (soma, média, etc.).

### **1.4. Google Data Studio**

O Data Studio é uma ferramenta *online* e gratuita de visualização interativa de dados da Google. Para ter acesso a ferramenta, basta ter um conta Google e clicar neste [link](https://datastudio.google.com/). As duas principais ferramentas são:

 - **Reports** para criar painéis de dados;
 - **Data source** para criar conectar painéis a fontes de dados.

## 2\. Dados

Neste módulo, vamos analisar dados sobre o a população mundial, em milhares de habitantes, entre os anos de 1950 e 2021. O conjunto de dados está neste [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/world-population.csv) e foi inspirado no conjunto de dados (compilados pelas nações unidas, sobre o tema do Kaggle, presente neste [link](https://www.kaggle.com/ahmethoso/wpp-population-by-age-and-sex?select=PopulationByAgeSex.csv). O painel de dados é o destino final dos dados, logo, todo o processo de manipulação deve ser feito a priori.

In [1]:
import pandas as pd
from datetime import datetime

### **2.1. Extração**

Nesta etapa, vamos extrair os dados da fonte de dados original e avaliar a sua qualidade.

In [2]:
!wget -q 'https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/world-population.csv' -O 'world_population.csv'

In [3]:
data = pd.read_csv('world_population.csv', sep=',', parse_dates=[2], infer_datetime_format=True)

In [4]:
data.head()

,id,location,time,child,adolescent,adult,old,male,female,total
0,1,Afghanistan,1950-01-01,3179.867,783.732,3398.243,390.275,4099.243,3652.874,7752.117
1,2,Afghanistan,1951-01-01,3235.425,790.341,3418.766,395.619,4134.756,3705.395,7840.151
2,3,Afghanistan,1952-01-01,3281.624,801.420,3452.410,400.542,4174.450,3761.546,7935.996
3,4,Afghanistan,1953-01-01,3327.335,814.914,3493.155,404.280,4218.336,3821.348,8039.684
4,5,Afghanistan,1954-01-01,3376.382,829.684,3538.851,406.399,4266.484,3884.832,8151.316


In [5]:
data.shape

(14040, 10)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14040 entries, 0 to 14039
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          14040 non-null  int64         
 1   location    14040 non-null  object        
 2   time        14040 non-null  datetime64[ns]
 3   child       14040 non-null  float64       
 4   adolescent  14040 non-null  float64       
 5   adult       14040 non-null  float64       
 6   old         14040 non-null  float64       
 7   male        14040 non-null  float64       
 8   female      14040 non-null  float64       
 9   total       14040 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 1.1+ MB


### **2.2. Transformação**

Vamos então aplicar uma série de transformações para limpar e pre-processar a base de dados.

 - Descartar a coluna `id`:

In [7]:
data = data.drop(['id'], axis=1)

 - Renomear a coluna `location` para `country` e `time` para `date`:

In [8]:
data = data.rename(columns={'location': 'country', 'time': 'date'})

 - Ajustar a coluna `date` para o [formato](https://support.google.com/datastudio/answer/6401549?hl=en#zippy=%2Cin-this-article) do Google Data Studio:

In [9]:
data['date'] = data['date'].apply(lambda num: datetime.strftime(num, '%b %d, %Y'))

 - Adicionar valores relativos as colunas de população parcial:

In [10]:
data['child_perc'] = 100 * data['child'] / data['total']
data['adolescent_perc'] = 100 * data['adolescent'] / data['total']
data['adult_perc'] = 100 * data['adult'] / data['total']
data['old_perc'] = 100 * data['old'] / data['total']

In [11]:
data['male_perc'] = 100 * data['male'] / data['total']
data['female_perc'] = 100 * data['female'] / data['total']

 - Arredondar todas as colunas de população para duas casas decimais:

In [12]:
for col in data.columns:
  if col not in ['date', 'country']:
    data[col] = data[col].apply(lambda num: round(num, 2))

 - Reorganizar as colunas:

In [13]:
data = data[['date', 'country', 'total', 'male', 'male_perc', 'female',	'female_perc', 'child', 'child_perc', 'adolescent', 'adolescent_perc', 'adult', 'adult_perc', 'old', 'old_perc']]

Vamos conferir o resultado final.

In [14]:
data.head()

,date,country,total,male,male_perc,female,female_perc,child,child_perc,adolescent,adolescent_perc,adult,adult_perc,old,old_perc
0,"Jan 01, 1950",Afghanistan,7752.12,4099.24,52.88,3652.87,47.12,3179.87,41.02,783.73,10.11,3398.24,43.84,390.27,5.03
1,"Jan 01, 1951",Afghanistan,7840.15,4134.76,52.74,3705.39,47.26,3235.43,41.27,790.34,10.08,3418.77,43.61,395.62,5.05
2,"Jan 01, 1952",Afghanistan,7936.00,4174.45,52.60,3761.55,47.40,3281.62,41.35,801.42,10.10,3452.41,43.50,400.54,5.05
3,"Jan 01, 1953",Afghanistan,8039.68,4218.34,52.47,3821.35,47.53,3327.34,41.39,814.91,10.14,3493.16,43.45,404.28,5.03
4,"Jan 01, 1954",Afghanistan,8151.32,4266.48,52.34,3884.83,47.66,3376.38,41.42,829.68,10.18,3538.85,43.41,406.40,4.99


### **2.3. Carregamento**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [15]:
data.to_csv('./world-population-wrangled.csv', sep=',', index=False)

## 3\. Relatório

Como dito na aula anterior, o relátorio ou painel de dados é o destino final dos dados. É atavés que pessoas interessadas no acompanhamento da atividade exercída vão se informar, em geral, pessoas menos técnicas e mais orientadas a negócio como analistas, gerentes e executivos. Portanto, um bom design de um painel (cores, tipografia, escolha e organização de elementos, etc.) é fundamental para transmitir a informação com qualidade.

### **3.1. EDA & KPIs**

Os paineis de dados buscam entregar duas funcionalidades para o usuário final:

1. o consumo de **indicadores chaves de desempenho** (*key performance indicator* ou KPI) consolidados;
1. a capacidade de praticar a **análise exploratória de dados** (*exploratory data analysis* ou EDA) interativa.

Em geral, o primeiro são métricas consolidadas que fazem parte do acompanhamento da atividade exercida, costumam aparecer no topo do painel. O restante do espaço útil é preenchido por elementos interativos (gráficos, mapas, etc.) que permitem o usuário a entender melhor uma dimensão específica.

### **3.2. Elementos**

As ferramentes de visualização de dados interativa possuem diversos elementos para a composição de um relatório como ferramentas gerais de edição de texto e imagens, gráficos, tabelas, cartões, etc. Em geral, a manipulação dos elementos se resume a interação entre três sub elementos:

 - Dimensões (colunas);
 - Métricas (agregações);
 - Filtros.

### **3.3. Compartilhamento**

Por fim, painéis de dados do Google Data Studio podem ser compartilhados na internet, assim como os cadernos do Kaggle.